#### Tenant Admin Settings

##### Data ingestion strategy:
<mark style="background: lightgreen;">**APPEND**</mark>

##### Related pipeline:

**Load_Tenant_Settings_E2E**

##### Source:

**Files** from FUAM_Lakehouse folder **bronze_file_location** variable

##### Target:

**1 Delta table** in FUAM_Lakehouse 
- **gold_table_name** variable value

In [ ]:
from datetime import datetime, timedelta
from pyspark.sql.functions import col, explode
from delta.tables import *

In [ ]:
## Parameters
display_data = False

In [ ]:
## Variables
bronze_file_location = f"Files/raw/tenant_settings/"
silver_table_name = "FUAM_Staging_Lakehouse.tenant_settings_silver"
gold_table_name = "tenant_settings"
gold_table_name_with_prefix = f"Tables/{gold_table_name}"

In [ ]:
# Clean Silver table, if exists
if spark.catalog.tableExists(silver_table_name):
    del_query = "DELETE FROM " + silver_table_name
    spark.sql(del_query)

In [ ]:
# Get Bronze data
bronze_df = spark.read.option("multiline", "true").json(bronze_file_location)

if display_data:
    display(bronze_df)


In [ ]:
# Explode json subset structure
# Error handling, because of API change which is not rolled out everywhere
try:
    exploded_df = bronze_df.select(explode("tenantSettings").alias("d"))
except:
    exploded_df = bronze_df.select(explode("value").alias("d"))

# Extract json objects to tabular form
silver_df = exploded_df.select(col("d.*"))

if display_data:
    display(silver_df)

In [ ]:
# Write prepared bronze_df to silver delta table
silver_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable(silver_table_name)

In [ ]:
# Get Silver table data
query = """
SELECT 
     to_date(current_timestamp()) AS TransferDate
     ,current_timestamp() AS TransferDateTime
     ,*
FROM """ + silver_table_name


silver_df = spark.sql(query)

if display_data:
     display(silver_df)

In [ ]:
# Merge semantic model refreshes to gold table
silver_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable(gold_table_name)

In [ ]:
# Write history of bronze files
mssparkutils.fs.cp(bronze_file_location, bronze_file_location.replace("Files/raw/", "Files/history/") + datetime.now().strftime('%Y/%m/%d') + "/", True)

In [ ]:
# Extract security group members
df_enabledSecurityGroups = spark.sql("""
SELECT settingName, TransferDateTime, enabledSecurityGroup.graphId , enabledSecurityGroup.name FROM (
SELECT settingName, TransferDateTime, EXPLODE(enabledSecurityGroups) enabledSecurityGroup 
FROM FUAM_Lakehouse.tenant_settings ) a
""")

df_enabledSecurityGroups.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("tenant_settings_enabled_security_groups")
